# eXamine Automation Tutorial

This case study demonstrates how to use the REST API of eXamine to study an annotated module in Cytoscape. The module that we study has 17 nodes and 18 edges and occurs within the KEGG mouse network consisting of 3863 nodes and 29293 edges. The module is annotated with sets from four different categories: (1) KEGG pathways and the GO categories (2) molecular process, (3) biological function and (4) cellular component.

There are three steps for visualizing subnetwork modules with eXamine. In the following, we will describe and perform the steps using the Automation functionality of Cytoscape. We refer to [tutorial.pdf](https://raw.githubusercontent.com/ls-cwi/eXamine/master/doc/tutorial/tutorial.pdf) for instructions using the Cytoscape GUI.

In [ ]:
# HTTP Client for Python
import requests

# Cytoscape port number
PORT_NUMBER = 1234

BASE_URL = "https://raw.githubusercontent.com/ls-cwi/eXamine/master/data/"

# The Base path for the CyRest API
BASE = 'http://localhost:' + str(PORT_NUMBER) + '/v1/'

#Helper command to call a command via HTTP POST
def executeRestCommand(namespace="", command="", args={}):
    postString = BASE + "commands/" + namespace + "/" + command
    res = requests.post(postString,json=args)
    return res

## Importing network and node-specific annotation

We start by importing the KEGG network directly from the eXamine repository on github.

In [ ]:
# First we import our demo network
executeRestCommand("network", "import url", {"indexColumnSourceInteraction":"1",
                                                   "indexColumnTargetInteraction":"2",
                                                   "url": BASE_URL + "edges.txt"})

We then import node-specific annotation directly from the eXamine repository on github. The imported file contains set membership information for each node. Note that it is important to ensure that set-membership information is imported as `List of String`, as indicated by `sl`.

In [ ]:
# Next we import node annotations
executeRestCommand("table", "import url",
                   {"firstRowAsColumnNames":"true",
                    "keyColumnIndex" : "1",
                    "startLoadRow" : "1",
                    "dataTypeList":"s,s,f,f,f,s,s,s,sl,sl,sl,sl",
                    "url": BASE_URL + "nodes_induced.txt"})

## Import set-specific annotation

We now describe how to import the set-specific annotations. In order to do so, eXamine needs to generate group nodes for each of the sets present in the module. To do so, we need to select nodes present in the module; these nodes have the value `small` in column `Module`, which we do as follows.

In [ ]:
executeRestCommand("network", "select", {"nodeList":"Module:small"})

Now that we have selected the nodes of the module, we can proceed with generating group nodes for each set (`Process`, `Function`, `Component` and `Pathway`).

In [ ]:
executeRestCommand("examine", "generate groups",
                   {"selectedGroupColumns" : "Process,Function,Component,Pathway"})

We import set-specific annotation, again directly from github.

In [ ]:
#Ok, time to enrich our newly greated group nodes with some interesting annotations
executeRestCommand("table", "import url",
                   {"firstRowAsColumnNames":"true",
                    "keyColumnIndex" : "1",
                    "startLoadRow" : "1",
                    "url" : BASE_URL + "sets_induced.txt"})

## Set-based visualization using eXamine

We now describe how to visualize the current selection. First, we set the visualization options.

In [ ]:
# Adjust the visualization settings
executeRestCommand("examine", "update settings",
                   {"labelColumn" : "Symbol",
                    "urlColumn" : "URL",
                    "scoreColumn" : "Score",
                    "showScore" : "true",
                    "selectedGroupColumns" : "Function,Pathway"})

We then select five groups.

In [ ]:
# Select groups for demarcation in the visualization
executeRestCommand("examine", "select groups",
                   {"selectedGroups":"GO:0008013,GO:0008083,mmu04070,mmu05200,mmu04520"})

There are two options: either we launch the interactive eXamine visualization, or we directly generate an SVG.

In [ ]:
# Launch the interactive eXamine visualization
executeRestCommand("examine", "interact", {})

The command below launches the eXamine window. If this window is blank, simply resize the window to force a redraw of the scene.

In [ ]:
# Export a graphic instead of interacting with it
# use absolute path; writes in Cytoscape directory if not changed 
executeRestCommand("examine", "export", {"path": "your-path-here.svg"})